# [빅데이터 분석과 응용] 기말 프로젝트
## 드라마 자원 추천 시스템
### MBA조 (이지희, 김나영, 강하연, 오현영)

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
import pandas as pd

In [2]:
from selenium.webdriver.chrome.options import Options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('C:\\Users\\User\\chromedriver.exe',chrome_options=chrome_options)

C:\Users\User\AppData\Local\Temp\ipykernel_21136\1780841689.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\User\\chromedriver.exe',chrome_options=chrome_options)
C:\Users\User\AppData\Local\Temp\ipykernel_21136\1780841689.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:\\Users\\User\\chromedriver.exe',chrome_options=chrome_options)


In [3]:
Title_english = []
Title_korean = []
Genre = []
Synopsis = []
Rating = []
Actor = []
Director = []
Writer = []
Tags = []
Watchers = []
Images = []

In [4]:
def get_title(soup): #Title_english
    div = soup.find("div", class_ = "box-header box-navbar mdl-component")
    myTitle = div.find("a")['title']
    return myTitle

In [5]:
def get_rate(soup): #Rating
    div = soup.find("div", class_ = "hfs")
    rate = div.find("b").get_text()
    return rate

In [6]:
def get_synopsis(soup): #Synopsis
    div = soup.find("div", class_ = "show-synopsis")
    synopsis = div.find("span").get_text()
    return synopsis

In [7]:
def get_watchers(soup):
    div = soup.find("div", class_ = "col-xs-8 col-sm-9 col-lg-9")
    div_1 = div.find_all("div", class_ = "hfs")
    watchers_pre = div_1[1].get_text()
    watchers_pre = watchers_pre.replace(",", "")
    watchers = int(re.findall('\d+', watchers_pre)[0])
    return watchers

In [8]:
def get_title_native(soup): #Title_korean
    div = soup.find("div", class_ = "show-detailsxss")
    li = div.find("li", class_ = "list-item p-a-0")
    myTitle = li.find("a")['title']
    return myTitle

In [9]:
def get_genre(soup): #Genre
    gen = [] #Genre
    div = soup.find("div", class_ = "show-detailsxss")
    li = div.find("li", class_ = "list-item p-a-0 show-genres")

    genre = None
    try:
        genre = li.find_all("a", class_ = "text-primary")
        for i in range(len(genre)):
            gen.append(genre[i].get_text())
    except:
        pass

    return gen

In [10]:
def get_tags(soup):
    tags = [] #Tags
    div = soup.find("div", class_ = "show-detailsxss")
    li = div.find("li", class_ = "list-item p-a-0 show-tags")
    span = li.find_all("span")
    for i in range(len(span)):
        tag = span[i].find("a", class_ = "text-primary")
        tags.append(tag.get_text())
    return tags

In [11]:
def get_director(soup): #Director
    div = soup.find("div", class_ = "show-detailsxss")
    li = div.find_all("li", class_ = "list-item p-a-0")

    myDirector = None 
    try:
        myDirector = li[3].find("a", class_ = "text-primary").get_text()
    except:
        pass
    
    return myDirector

In [12]:
def get_writer(soup): #Writer
    div = soup.find("div", class_ = "show-detailsxss")
    li = div.find_all("li", class_ = "list-item p-a-0")
    
    myWriter = None
    try:
        myWriter = li[2].find("a", class_ = "text-primary").get_text()
    except:
        pass
    
    return myWriter

In [13]:
def get_actor(soup):
    actors = [] #Actor
    ul = soup.find_all("ul", class_ = "list no-border p-b clear")
    
    mainActor = None
    supportActor = None
    try:
        mainActor = ul[2]
        supportActor = ul[3]
    except:
        pass

    for ac in (mainActor, supportActor):
        if ac is None:
            continue

        div = ac.find_all("div", class_ = "col-xs-9 col-sm-8 p-a-0")
        for i in range(len(div)):
            actor = div[i].find("a", class_ = "text-primary").get_text()
            actors.append(actor)
    return actors

In [14]:
def get_img(soup):
    images = driver.find_elements_by_css_selector("#content > div > div.container-fluid.title-container > div > div.col-lg-8.col-md-8.col-rightx > div:nth-child(1) > div.box-body > div > div.col-sm-4.film-cover.cover > a.block > img")
    for image in images:
        url = image.get_attribute('src')
    return url

In [15]:
#아이디 리스트 가져오기
pattern = re.compile('"mdl-\d{5}"')

def get_id(soup):
    id_list_temp = []
    id_list = []
    pattern = re.compile('"mdl-\d{5}"')
    div = soup.find("div", class_ = "m-t nav-active-border b-primary")
    div = div.find_all("div", class_="box")
    
    for i in range(len(div)):
        str_div = str(div[i])
        id_list_temp.append(re.findall(pattern, str_div))
        
    for text in id_list_temp:
        text = str(text)
        id_list.append(re.sub(r'[\D]','', text))
    
    return id_list

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [17]:
pre = "https://mydramalist.com/search?adv=titles&ty=68&co=3&st=3&so=top&page="
for i in range(1, 161):
    urls = pre+str(i) #랭킹 전체 페이지
    res = requests.get(urls)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    driver.get(urls)
    driver.maximize_window()
    driver.implicitly_wait(3)
    
    id_list = get_id(soup)
    
    for text in id_list:
        if text == '':
            continue
        else:
            #랭킹별 하나의 드라마 정보 페이지
            time.sleep(1)
            css_selector = "#mdl-"+text+" > div > div > div.col-xs-9.row-cell.content > h6 > a:nth-child(1)"
            element = driver.find_element_by_css_selector(css_selector)
            element.send_keys(Keys.ENTER)
            
            url = driver.current_url
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            
            Title_english.append(get_title(soup))
            Rating.append(get_rate(soup))
            Genre.append(get_genre(soup))
            Title_korean.append(get_title_native(soup))
            Synopsis.append(get_synopsis(soup))
            Director.append(get_director(soup))
            Writer.append(get_writer(soup))
            Tags.append(get_tags(soup))
            Watchers.append(get_watchers(soup))
            Images.append(get_img(soup))
            
            #cast&crew 메뉴 클릭
            time.sleep(1)
            css_selector = "#content > div > div.container-fluid.title-container > div > div.col-lg-8.col-md-8.col-rightx > div:nth-child(1) > div.box-header.box-navbar.mdl-component > ul > li:nth-child(3) > a"
            element = driver.find_element_by_css_selector(css_selector)
            element.send_keys(Keys.ENTER)
            
            url = driver.current_url
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            
            Actor.append(get_actor(soup))
            
        time.sleep(1)
        driver.get(urls)

    print(i)
driver.quit()
print("전체 크롤링에 성공하였습니다.")

C:\Users\User\AppData\Local\Temp\ipykernel_21136\2767589282.py:20: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector(css_selector)
C:\Users\User\AppData\Local\Temp\ipykernel_21136\1565661478.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  images = driver.find_elements_by_css_selector("#content > div > div.container-fluid.title-container > div > div.col-lg-8.col-md-8.col-rightx > div:nth-child(1) > div.box-body > div > div.col-sm-4.film-cover.cover > a.block > img")
C:\Users\User\AppData\Local\Temp\ipykernel_21136\2767589282.py:41: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector(css_selector)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
전체 크롤링에 성공하였습니다.


In [18]:
craw_df = pd.DataFrame([Title_english, Rating, Genre, Title_korean, Synopsis, Actor, Director, Writer, Tags, Images, Watchers ]).T

craw_df.to_csv('Drama_List.csv', sep=',')

In [19]:
craw_df

,0,1,2,3,4,5,6,7,8,9,10
0,Move to Heaven,9.2,"[Life, Drama, Family]",무브 투 헤븐: 나는 유품정리사입니다,Han Geu Roo is a 20-year-old with Autism. He w...,"[Lee Je Hoon, Tang Jun Sang, Hong Seung Hee, J...",Kim Sung Ho,Yoon Ji Ryun,"[Uncle-Nephew Relationship, Autism, Death, Sav...",https://i.mydramalist.com/Rle36_4c.jpg?v=1,47482
1,Hospital Playlist,9.1,"[Friendship, Romance, Life, Medical]",슬기로운 의사 생활,The stories of people going through their days...,"[Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Ki...",Lee Woo Jung,Shin Won Ho,"[Multiple Mains, Best Friends, Hospital, Slow ...",https://i.mydramalist.com/RXXL6_4c.jpg?v=1,68688
2,Flower of Evil,9.1,"[Thriller, Romance, Crime, Melodrama]",악의 꽃,Although Baek Hee Sung is hiding a dark secret...,"[Lee Joon Gi, Moon Chae Won, Jang Hee Jin, Seo...",Yoo Jung Hee,Kim Chul Gyu,"[Married Couple, Deception, Family Secret, Sus...",https://i.mydramalist.com/WAEAp_4c.jpg?v=1,73946
3,Hospital Playlist 2,9.1,"[Friendship, Romance, Life, Medical]",슬기로운 의사생활++,Everyday is extraordinary for five doctors and...,"[Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Ki...",Lee Woo Jung,Shin Won Ho,"[Workplace, Strong Friendship, Best Friends, M...",https://i.mydramalist.com/dKY0d_4c.jpg?v=1,34295
4,My Mister,9.1,"[Psychological, Life, Drama, Family]",나의 아저씨,Park Dong Hoon is a middle-aged engineer who i...,"[Lee Sun Kyun, IU, Park Ho San, Song Sae Byuk,...",Park Hae Young,Kim Won Suk,"[Nice Male Lead, Strong Female Lead, Smart Fem...",https://i.mydramalist.com/2lb8k_4c.jpg?v=1,51608
...,...,...,...,...,...,...,...,...,...,...,...
1776,That's Perfect,7.5,"[Romance, Sitcom]",딱좋아!,A sitcom revolving around the friendships and ...,[],None,None,"[College Life, Youth, Friendship]",https://i.mydramalist.com/XdqRXc.jpg?v=1,62
1777,That Woman Catches People,7.5,"[Romance, Drama, Family, Melodrama]",그 여자 사람잡네,"The story of two women, Baek Sang Ah and Han B...",[Son Min Woo],None,None,[],https://i.mydramalist.com/l0qX6c.jpg?v=1,31
1778,Golden Tower,8.0,"[Comedy, Family]",황금거탑,"Based in a farming village, this drama deals w...","[Lee Yong Joo, Kim Jae Woo, Baek Bong Ki, Choi...",Jo Seung Hee,Min Jin Gi,"[Farming, Farmer, Agriculture, Farm Life]",https://i.mydramalist.com/g0RzWc.jpg?v=1,134
1779,Tomorrow Love,9.0,"[Friendship, Life, School, Youth]",내일은 사랑,Focuses on campus students' friendship and life.,"[Lee Byung Hun, Park So Hyun, Kim Jung Kyoon, ...",Yoon Seok Ho,Son Young Mok,[],https://i.mydramalist.com/rNZEEc.jpg?v=1,34
